In [ ]:
import requests, os, json
ALCHEMY_KEY = os.getenv('ALCHEMY_KEY')
MAKER_KEY = os.getenv('MAKER_KEY')
MAKER_EVENT_NAME = os.getenv('MAKER_EVENT_NAME')
TOPIC = os.getenv('TOPIC', 'bluemix')

MAKER_EVENT_URL = 'https://maker.ifttt.com/trigger/{event}/with/key/{key}'
MEETUP_URL = 'http://www.meetup.com/find/events/?allMeetups=false&keywords={topic}&radius=50&userFreeform={city}&change=yes&eventFilter=mysugg'

In [ ]:
def send_maker_request(username, location, meetup_url):
    post_url = MAKER_EVENT_URL.format(event=MAKER_EVENT_NAME, key=MAKER_KEY)
    values = dict(value1=username, value2=location, value3=meetup_url)
    r = requests.post(post_url, data=json.dumps(values), headers={'Content-Type': 'application/json'}, timeout=90)
    r.raise_for_status()
    
def send_alchemy_request(tweet):
    resp = requests.get('http://gateway-a.watsonplatform.net/calls/text/TextGetRankedNamedEntities', params=dict(
            apikey=ALCHEMY_KEY,
            text=tweet,
            outputMode='json',
            timeout=10
    ))
    resp.raise_for_status()
    return resp

In [ ]:
# GET /_echo
print(REQUEST)

In [ ]:
# GET /_city
# Just run the 'text' parameter through the Alchemy entity extraction
resp = send_alchemy_request(json.loads(REQUEST)['args']['text'])
print(resp.text)

In [ ]:
# GET /generate_search_url
# Find the city's name by running the 'text' parameter through the Alchemy entity extraction
req = json.loads(REQUEST)
resp = send_alchemy_request(req['args']['text'])
named_entities = resp.json()['entities']
cities = [
    entity['text'] for entity in named_entities
        if entity['type'] == 'City'
]
users = req['args']['user']
# If a city was found, trigger the Maker event with the querying username and search URL
if len(cities) > 0 and len(users) > 0:
    username = users[0]
    mentioned_city = cities[0]
    url = MEETUP_URL.format(city=mentioned_city, topic=TOPIC)
    send_maker_request('@'+username, mentioned_city, url)
